# X2K_Web Genetic Algorithm

## X2K Web API

In [ ]:
import pandas as pd
import http.client
import json

all_x2k_options = {
    'TF-target gene background database used for enrichment': [
        'ChEA 2015',
        'ENCODE 2015',
        'ChEA & ENCODE Consensus',
        'Transfac and Jaspar',
        'ChEA 2016',
        'ARCHS4 TFs Coexp',
        'CREEDS',
        'Enrichr Submissions TF-Gene Coocurrence',
    ],
    'kinase interactions to include': [#kea 2016
        'kea 2018',
        'ARCHS4',
        'iPTMnet',
        'NetworkIN',
        'Phospho.ELM',
        'Phosphopoint',
        'PhosphoPlus',
        'MINT',
    ],
    'enable_ppi': [
        'ppid',
        'Stelzl',
        'IntAct',
        'MINT',
        'BioGRID',
        'Biocarta',
        'BioPlex',
        'DIP',
        'huMAP',
        'InnateDB',
        'KEGG',
        'SNAVI',
        'iREF',
        'vidal',
        'BIND',
        'figeys',
        'HPRD',
    ],
    'max_number_of_interactions_per_article':  {"10":15, "01":50, "11":200, "00":1000000},
    'max_number_of_interactions_per_protein': {"10":50, "01":100, "11":200, "00":500},
    'min_network_size': {"10":1, "01":10, "11":50, "00":100},
    'min_number_of_articles_supporting_interaction': {"10":0, "01":1, "11":5, "00":10},
    'path_length': {"0":1, "1":2},
    'included organisms in the background database': {"10": "human", "01": "mouse", "11": "both", "00": "RESHUFFLE"},
}

def run_X2K(input_genes, x2k_options={}):
    # Open HTTP connection
    conn = http.client.HTTPConnection("amp.pharm.mssm.edu") #
    #conn = http.client.HTTPConnection("localhost:8080", timeout=20)
    # Get default options
    default_options = {'text-genes': '\n'.join(input_genes), 'included_organisms': 'both', 'included_database': 'ChEA 2015',
                       'path_length': 2, 'minimum network size': 50, 'min_number_of_articles_supporting_interaction': 2,
                       'max_number_of_interactions_per_protein': 200, 'max_number_of_interactions_per_article': 100,
                       'biocarta': True, 'biogrid': True, 'dip': True, 'innatedb': True, 'intact': True, 'kegg': True, 'mint': True,
                       'ppid': True, 'snavi': True, 'number_of_results': 50, 'sort_tfs_by': 'combined score', 'sort_kinases_by': 'combined score',
                       'kinase interactions to include': 'kea 2018'}
    # Update options
    for key, value in x2k_options.items():
        if key in default_options.keys() and key != 'text-genes':
            default_options.update({key: value})
    # Get payload
    boundary = "----WebKitFormBoundary7MA4YWxkTrZu0gW"
    payload = ''.join(['--'+boundary+'\r\nContent-Disposition: form-data; name=\"{key}\"\r\n\r\n{value}\r\n'.format(**locals()) for key, value in default_options.items()])+'--'+boundary+'--'
    # Get Headers
    headers = {
        'content-type': "multipart/form-data; boundary="+boundary,
        'cache-control': "no-cache",
    }
    # Initialize connection
    conn.request("POST", "/X2K/api", payload, headers)
    # Get response
    res = conn.getresponse()
    # Read response
    data = res.read().decode('utf-8')
    # Convert to dictionary
    x2k_results = {key: json.loads(value) if key != 'input' else value for key, value in json.loads(data).items()}
    # Clean results
    x2k_results['ChEA'] = x2k_results['ChEA']['tfs']
    x2k_results['G2N'] = x2k_results['G2N']['network']['nodes']
    x2k_results['KEA'] = x2k_results['KEA']['kinases']
    x2k_results['X2K'] = x2k_results['X2K']['network']
    # Return results
    return x2k_results

In [ ]:
import os

# Standardize genes to HGNC symbols
mapping = pd.read_table('../X2K_Summaries/General_Resources/Moshe_mapping/mappingFile_2017.txt', header=None)
greekLetters = pd.read_csv('../X2K_Summaries/General_Resources/GreekLetter_Converter.csv', names=['Greek', 'Abbrev'], header=0 )
greekLetters = greekLetters.apply(lambda x: x.str.strip('\xa0'))

def standardizeGeneSymbol(gene):
    if gene.__contains__('AURORA'):
        HGNC = 'AURK' + gene[-1]
    elif any(substring in gene for substring in greekLetters['Greek']):
        for letter in greekLetters['Greek']:
            LETTER = letter.upper()
            if gene.__contains__(LETTER):
                HGNC = gene.replace(LETTER, greekLetters.loc[greekLetters['Greek']==letter,'Abbrev'].values[0] )
    else:
        HGNC = gene
    if HGNC in mapping[0]:
        HGNC = mapping.iloc[mapping[0]==HGNC, 1] 
    return HGNC


def parse_GEO_line(line):
    lineSp = line.split('\t')
    expt_name = lineSp[0]
    genes = [str(x.strip(',1.0')) for x in lineSp[2:-1]]
    return expt_name, genes

def prepare_options_for_x2k(input_genes, x2k_parameters):
    options=x2k_parameters.copy()
    for param in options:
        options[param] = options[param]['selection']
    # Add input_genes
    options['text-genes'] = input_genes
    # Convert ppi into enable flags
    for ppi in options['enable_ppi']:
        options['enable_' + ppi] = 'true'
    del options['enable_ppi']
    # Convert any lists
    return {
        k: '\n'.join(v) if type(v) == list else str(v)
        for k, v in options.items() 
    }

def translateDatabases(binaryString_segment, _dbs):
    selection = []
    for i, bit in enumerate(binaryString_segment):
        if bit == "1":
            selection.append(_dbs[i])
    return selection

def binary_to_parameters(binaryString):
    x2k_parameters={}
    stringCount=0
    for key in all_x2k_options:
        parameter = all_x2k_options[key]
        if type(parameter)==list:
            bitSegment = binaryString[stringCount:stringCount + len(parameter)]
            selection = translateDatabases(bitSegment, parameter)
            x2k_parameters[key] = {'selection':selection, 'bits':bitSegment}
            stringCount += len(bitSegment)
        if type(parameter)==dict:
            bitLength = len(list(parameter.keys())[0])
            bits = binaryString[stringCount:stringCount + bitLength]
            selection = parameter[bits]
            x2k_parameters[key] = {'selection':selection, 'bits':bits}
            stringCount += bitLength
    return x2k_parameters

def parameters_to_binary(x2k_parameters):
    newBinary=[]
    for key in all_x2k_options:
        newBinary.append( x2k_parameters[key]['bits'] )
    return ''.join(newBinary)

"""
params = binary_to_parameters(binaryString)
recoveredBinary = parameters_to_binary(params)
binaryString == recoveredBinary
"""

def reshuffle_binary(binaryString):
    import random
    new_options = binary_to_parameters(binaryString)
    for param in new_options:
        selection = new_options[param]['selection']
        while selection == "RESHUFFLE":
            selection = random.choice( list(all_x2k_options[param].values()) )
            new_options[param]['selection'] = selection
    newBinary = parameters_to_binary(new_options)
    return newBinary
    
 
############ Parallel processing of X2K ############

    # Correct binaryString

def prepare_all_x2k_inputs(newBinary, gmtLimit=False):
    # Open GMT
    with open('Genetic_Algorithm/testgmt/'+os.listdir('Genetic_Algorithm/testgmt')[0]) as gmt_file:
        gmtLines = gmt_file.readlines()
    if gmtLimit!=False:
        gmtLines = gmtLines[0:gmtLimit]
    # Prepare inputs
    def prepare_one_x2k_input(line, newBinary):
        expt_name, input_genes = parse_GEO_line(line)
        ## Standardize input genes
        input_genes = [standardizeGeneSymbol(g) for g in input_genes]
        x2k_parameters = binary_to_parameters(newBinary)
        x2k_options = prepare_options_for_x2k(input_genes, x2k_parameters)
        x2k_input = {'options':x2k_options, 'expt_name':expt_name}
        return x2k_input
    all_x2k_inputs = list(map(prepare_one_x2k_input, gmtLines, [newBinary]*len(gmtLines)))
    return all_x2k_inputs

def run_X2K_once(x2k_input):
    x2k_options = x2k_input['options']
    input_genes = x2k_options['text-genes'].split('\n')
    expt_name = x2k_input['expt_name']
    try:
        x2k_results = run_X2K(input_genes=input_genes, x2k_options=x2k_options)
    except:
        expt_name='FAIL'; x2k_results='NA'
    return {'experiment':expt_name,'results':x2k_results}

def parallel_x2k_results(all_x2k_inputs, threadPool_size=20):
    # ********* Parallelize X2K across experiments ********* #
    # ****************************************************** #
    # import time
    # start = time.time()
    from multiprocessing.dummy import Pool as ThreadPool
    pool = ThreadPool(threadPool_size)
    raw_x2k_results = pool.map(run_X2K_once, all_x2k_inputs)
    pool.close() 
    pool.join() 
    # end = time.time()
    # print(end - start)
    
    # Post-process pooled results
    allExpts_x2k_results={}
    for dict in raw_x2k_results:
        expt_name = dict['experiment']
        if expt_name!='FAIL':
            allExpts_x2k_results[expt_name] = dict['results']  
   
    return allExpts_x2k_results
"""
binaryString=createPopulation(1)[0]
population=createPopulation(10)
"""

## Run X2K GA

### GA Support Functions

In [ ]:
###################################
# 0. Create initial population
###################################
def stringLength():
    string_length=0
    for key in all_x2k_options:
        parameter = all_x2k_options[key]
        if type(parameter) ==list:
            string_length+=len(parameter) # All possible databases
        if type(parameter)==dict:
            string_length += len(next(iter(parameter.keys()))) # Random bit option
    return  string_length

def createPopulation(popSize):
    from random import choice
    binaryStringLength = stringLength()
    populationinit = []
    for i in range(popSize):
        populationinit.append(''.join(choice(('0', '1')) for _ in range(binaryStringLength)) )
        print(populationinit[i])
    return populationinit

###################################
# 1. Calculate fitness
###################################
def pvalue_matrix(all_x2k_results, dataType='KEA' ):
    nameKey = {'ChEA':'simpleName','KEA':'name', 'G2N':'name'}
    ## dict_keys(['X2K', 'ChEA', 'KEA', 'G2N', 'input', 'Experiment', 'x2k_options', 'binaryString'])
    # Experiment -> Kinase -> kinase results
    pvalDict={} 
    for expt in all_x2k_results:
        results = all_x2k_results[expt][dataType]
        if dataType == 'G2N':
            for g in results:
                g['name'] = g['name'].split("-")[0]
        predictedKinases = [y[nameKey[dataType]] for y in results]
        predictedPvals = [y['pvalue'] for y in results]
        # if replaceNAs==True:
        #     predictedPvals = [1.0 if math.isnan(x) else x for x in predictedPvals]
        pvalDict[expt] = dict(zip(predictedKinases, predictedPvals))
     return pvalDict

# Calculate fitness every each individual in pop
def population_fitness(gen, population, fitnessDict, fitness_method='target_shuffled_difference', gmtLimit=False, threadPool_size=20):
    population_results={}
    newFitnessDict = fitnessDict.copy()
    for i,binaryString in enumerate(population):
        unique_id = "ind"+str(i)+"_gen"+str(gen)
        newBinary = reshuffle_binary(binaryString)
        # Pull from fitnessDict if newBinary previously seen
        if newBinary in newFitnessDict:
            print("Pulling info from fitnessDict")
            dict_pull = newFitnessDict[newBinary].copy()
            dict_pull['generation'] = gen
            population_results[unique_id] = dict_pull
            print(unique_id+": Fitness = "+str(round(dict_pull['fitness'],3))+" [pulled from fitnessDict]")
        else:
            all_x2k_inputs = prepare_all_x2k_inputs(newBinary, gmtLimit)
            all_x2k_results = parallel_x2k_results(all_x2k_inputs=all_x2k_inputs, threadPool_size=threadPool_size)
            CHEA_pvalDict = pvalue_matrix(all_x2k_results, 'ChEA')
            KEA_pvalDict = pvalue_matrix(all_x2k_results, 'KEA')
            G2N_pvalDict = pvalue_matrix(all_x2k_results, 'G2N')
            fitness, shuffled_ranks, target_ranks = eval(fitness_method)(KEA_pvalDict)
            population_results[unique_id] = {'generation':gen, 'binaryString':binaryString, 'newBinary':newBinary,
                                             'fitness':fitness, 'KEA_results':KEA_pvalDict, 'CHEA_results':CHEA_pvalDict,
                                             'G2N_results':G2N_pvalDict,
                                             'shuffled_ranks':shuffled_ranks, 'target_ranks':target_ranks}
            print(unique_id+": Fitness = "+str(round(fitness,3)))
            #Add to fitness dictionary
            newFitnessDict[newBinary] = population_results[unique_id]
    return population_results, newFitnessDict
"""
population=createPopulation(10)
pop_fitness_results, newFitnessDict = population_fitness(gen=0, fitnessDict={}, population=population, gmtLimit=15)
"""

###################################
# 2. Select fittest individuals
###################################
def selectFittest(topNum, pop_fitness_results, selection_method='Fitness-proportional'):
    import pandas as pd
    fitDF = pd.DataFrame(pop_fitness_results).T
    import pandas as pd
    def print_fittest(fittestDF):
        fitnesses = fittestDF["fitness"].values
        print('Top fitnesses: '+str([round(x,3) for x in fitnesses]))
    
    if selection_method == 'Fitness-proportional':
        fittestDF = fitDF.sort_values(by='fitness', ascending=False).iloc[:topNum,:].copy()
        print_fittest(fittestDF)
    # Tournament selection (less stringent)
    ## Split the population into equal subgroups, and then select the fittest individual from each group
    if selection_method == 'Tournament':
        fittestDF=pd.DataFrame()
        if fitDF.shape[0] % topNum!=0:
            print("Tournament selection requires that populationSize/topNum and childrenPerGeneration/topNum are both whole numbers.")
        subsetSize = int( fitDF.shape[0] / topNum )
        for t in range(topNum):
            subDF = fitDF.sample(n=subsetSize, replace=False)
            fittestDF = fittestDF.append( subDF.sort_values(by=['fitness'], ascending=False).iloc[0,:].copy())
        print_fittest(fittestDF)

    if selection_method == 'mixedTournament':
        if fitDF.shape[0]%topNum!=0 or topNum%2!=0:
            print("WARNING:: Tournament selection requires that populationSize/topNum, \n"
                  +"childrenPerGeneration/topNum, and topNum/2 to be whole numbers.")
        topNumHalf = int(topNum/2)
        sortedDF = fitDF.sort_values(by=['fitness'], ascending=False).copy()
        # The first half of the new pop are the fittest parents overall
        fittestDF = sortedDF.iloc[:topNumHalf, :].copy()
        # Then run Tournament selection on the rest of the population to get the other half of the new pop
        everybodyElse = sortedDF.iloc[topNumHalf:, :].copy()
        subsetSize = int(everybodyElse.shape[0] / topNumHalf)
        for t in range(topNumHalf):
            subDF = everybodyElse.sample(n=subsetSize, replace=False)
            fittestDF = fittestDF.append(subDF.sort_values(by=['fitness'], ascending=False).iloc[0, :].copy())
        print_fittest(fittestDF)
    else:
        print("Use viable 'selectionMethod'")
    return fittestDF

"""
fittestDF = selectFittest(topNum=4, pop_fitness_results=pop_fitness_results, selection_method='Fitness-proportional')
"""

###################################
# 3. Crossover/breed fittest
###################################
###################################
# 4. Introduce random mutations
###################################
def createChild(individual1, individual2, crossover_points, crossover_locations="evenlyDistributed"):
    if crossover_locations=="evenlyDistributed":
        chunkSize = int(len(individual1) / (crossover_points+1))
        ind1Split = [individual1[i:i + chunkSize] for i in range(0, len(individual1), chunkSize)]
        ind2Split = [individual2[i:i + chunkSize] for i in range(0, len(individual2), chunkSize)]
    elif crossover_locations=='random':
        from random import sample
        cutpoints = sorted(sample(range(1, len(individual1)-1), crossover_points)) # randomly generate n non-overlapping numbers
        def splitParent(parent, cutpoints):
            indSplit=[]
            for i,num in enumerate(cutpoints):
                #print("**Cutpoint index= "+str(i))
                if i == 0: # If it's the first cutpoint, take all values up to the first index+1
                    start = 0
                    end = num
                else:
                    start = cutpoints[i-1]
                    end = num
                segment = parent[start:end]
                #print("Cutpoint= " + str(start) + " : " + str(end))
                #print("------- "+segment+" -------")
                indSplit.append(segment)
            # Add the very last segment
            indSplit.append(parent[cutpoints[-1]:])
            return indSplit
        ind1Split = splitParent(individual1, cutpoints)
        ind2Split = splitParent(individual2, cutpoints)
    # Put together the new child
    from random import random
    childFragments=[]
    for fragment in range(len(ind1Split)):
        if int(100*random()) < 50: # Just randomly picks from ParentA or ParentB for each individual parameter
            childFragments.append(ind1Split[fragment])
        else:
            childFragments.append(ind2Split[fragment])
    child = "".join(childFragments)
    return child
"""
individual1, individual2 = fittestDF['newBinary'][:2,]
child = createChild(individual1, individual2, crossoverPoints=8, crossoverLocations="evenlyDistributed")
"""

def mutateChild(child, mutationRate):
    from random import random
    mutant = ''
    for bit, val in enumerate(child):
        rando = random()
        if rando <= mutationRate and val == '1':
            mutant = str(str(mutant) + '0')
        elif rando <= mutationRate and val == '0':
            mutant = str(str(mutant) + '1')
        else:
            mutant = str(str(mutant) + str(val))
    return mutant

def createChildren(numberOfChildren, fittestDF, mutationRate, crossover_points, crossover_locations):
    from random import random
    fittest = fittestDF['newBinary'].tolist()
    #breedingChances = []
    # Add noise to fitness score?
    # for b in range(len(Fittest)):
    #     breedingChances.append(np.random.uniform(1 + breedingVariation, 1 - breedingVariation) * int(fittestFitness[b]))
    #     topBreeders = [x for _, x in sorted(zip(breedingChances, Fittest), reverse=True)]
    # Breed n times
    # 'Once you're in, you're in'. After selecting the top fittest individuals, it doesn't matter who is fitter within that group: everyone breeds with everyone else randomly
    children = []
    for i in range(numberOfChildren):
        ind1 = int(random()*len(fittest))
        ind2 = int(random()*len(fittest))
        child = createChild(fittest[ind1], fittest[ind2], crossover_points, crossover_locations)
        # MUTATE the children!
        child = mutateChild(child, mutationRate)
        children.append(child)
    return children
"""
children = createChildren( 8, fittestDF, 0.01, crossoverLocations="evenlyDistributed")
"""

### Fitness Functions

In [ ]:
############################
# Fitness Support Functions
############################
def values_to_ranks(valuesDF, ascending=False):
    Ranks={}
    # assign ranks based on given value (could be pvalue, -log(pvalue), ranks, etc)
    for col in valuesDF:
        # Since zscore comes from -log(pvalue), flip the rank order so that low numbered ranks are still the best
        orderedCol = valuesDF[col].sort_values(ascending=ascending)
        # Shuffle order of 0s
        nonNAs = orderedCol.loc[~orderedCol.isna()]
        try:
            NAs = orderedCol.loc[orderedCol.isna()].sample(frac=1)
        except:
            NAs = pd.Series(dtype=float)
        shuffledCol = pd.concat([nonNAs, NAs])
        # Assign ranks
        newRanks = pd.Series(data=range(0,len(shuffledCol)), name=col, index=shuffledCol.index)
        newRanks.sort_index(inplace=True) # Sort by index
        Ranks[col] = dict(zip(newRanks.index, newRanks.values)) 
    return pd.DataFrame(Ranks)

import math
def values_to_zscores(DF, dropNAs=False):
    if dropNAs==True:
        # Drop all rows that ONLY have (0). Never appeared across any experiment
        # Keeping the all 0s messes up the zscore
        df = DF[(DF.T != math.nan).any()]
    else:
        df = DF.copy()
    zScoreDF = df.apply(lambda x: (x-x.mean()) / x.std(ddof=0) ) 
    return zScoreDF

def scaled_ranks(DFstack):
    scaledDF = DFstack.copy()
    scaledDF['Rank'] -= scaledDF['Rank'].min() 
    scaledDF['Rank'] /= scaledDF['Rank'].max()
    return scaledDF

def clearTestGMT():
    import os
    dir_name = "Genetic_Algorithm/testgmt/"
    files = os.listdir(dir_name)
    for item in files:
        if item.endswith(".txt") or item.endswith(".gmt"):
            os.remove(os.path.join(dir_name, item))

import numpy as np
def negLog(pval_matrix):
    negLog_matrix = -np.log(pval_matrix)
    return negLog_matrix
    
############################
# Fitness Functions
############################ 
#from scipy.stats import kendalltau
#from scipy.stats import spearmanr
#from scipy.stats import entropy
#from scipy.stats import kstest
#from sklearn.metrics import mutual_info_score
from scipy.stats import ranksums # Assumes sample independence. Use Wilcoxon signed-rank test instead when sample are dependent
# Kendall and Spearman calculate rank correlations (the difference between 2 rank distributions), 
##  but don't distinguish between one being lower that the other (need to minimize Target ranks AND maximize difference between Target and Shuffle).
## Can't use t-test because the distributions are often not normal.

# Select ranking method
def rank_stack(pval_dict, zscore, scaledRanks, standardize=True):
    pval_matrix = pd.DataFrame(pval_dict)
    negLog_matrix = negLog(pval_matrix)
    if zscore==True:
        zScores = values_to_zscores(negLog_matrix)
        ranks = values_to_ranks(zScores)
    else:
        ranks = values_to_ranks(negLog_matrix)
    
    DFstack = ranks.stack().reset_index()
    DFstack.columns = ['Kinase','Experiment','Rank']
    if scaledRanks==True:
        DFstack = scaled_ranks(DFstack)
    # Standardize target kinases in cols
    if standardize==True:
        DFstack['Experiment'] = ['_'.join([standardizeGeneSymbol(x.split('_')[0])]+x.split('_')[1:]) for x in  DFstack['Experiment']]
    return DFstack

def shuffle_targets(DFstack, DFstack_target):
    DFstack_shuffled = DFstack_target.copy()
    DFstack_shuffled.loc[:,['Experiment','Rank']] = DFstack.sample(n=len(DFstack_target)).loc[:,['Experiment','Rank']].values
    DFstack_shuffled.index = DFstack_shuffled['Kinase']
    return DFstack_shuffled

def stats_test(DFstack_shuffled, DFstack_target):
        ## Calculate the difference between two rank distributions
        #fitness = DFmerged['Rank_shuffled'].mean() - DFmerged['Rank_target'].mean()
        #kendalltau(DFmerged['Rank_shuffled'], DFmerged['Rank_target'])
        #spearmanr(DFmerged['Rank_shuffled'], DFmerged['Rank_target'])
        #fitness = entropy(pk=DFmerged['Rank_target_prob'], qk=DFmerged['Rank_shuffled_prob'])
        #kstest(DFmerged['Rank_shuffled'], DFmerged['Rank_target'])
        Wilcoxon_ranksums = ranksums(DFstack_shuffled['Rank'], DFstack_target['Rank'])
        stats_result = Wilcoxon_ranksums[0]
        return stats_result, DFstack_shuffled['Rank']

def target_shuffled_difference(KEA_pvalDict):
    # Iterate stats test
    stats_results=[]; all_shuffled_ranks=[]; all_target_ranks=[]
    #for i in range(100):
    def iterate_stats(KEA_pvalDict):
        DFstack = rank_stack(KEA_pvalDict, zscore=True, scaledRanks=True)
        # Target Kinases Only
        DFstack_target = DFstack.loc[DFstack['Kinase']==DFstack['Experiment'].str.split('_').str[0]] # Returning way too few
        # Shuffled
        DFstack_shuffled = shuffle_targets(DFstack, DFstack_target)
        
        stats_result, shuffled_ranks = stats_test(DFstack_shuffled, DFstack_target)
        stats_results.append(stats_result)
        all_shuffled_ranks.extend(shuffled_ranks.tolist())
        all_target_ranks.extend(DFstack_target['Rank'].tolist()) 
        #return {'stats_results':stats_results,'shuffled_ranks':all_shuffled_ranks,'target_ranks':all_target_ranks}
        
    # results = [iterate_stats(KEA_pvalDict) for x in range(100)]
    iterations=100
    map_run = list(map(iterate_stats, [KEA_pvalDict]*iterations )) 
        
    #all_shuffled_ranks = pd.DataFrame(all_shuffled_ranks).stack()
    fitness = sum(stats_results)/len(stats_results)
    return fitness, all_shuffled_ranks, all_target_ranks
        

"""
pval_dict = pvalue_matrix(all_x2k_results, 'KEA')
"""

### GA Function

In [ ]:
def X2K_Web_GA_train(GAset):
    # Prepare GMT input
    from shutil import copyfile
    clearTestGMT()
    GMT = GAset['GMT_train']
    copyfile(GMT, "Genetic_Algorithm/testgmt/"+ GMT.split("/")[-1])
    # Results Dicts
    all_GA_results={}
    fitnessDict={}
    # 0. Create initial population 
    population = createPopulation(GAset['initial_pop_size'])
    # Loop over n generations
    for gen in range(GAset['generations']):
        print('================ GENERATION '+str(gen)+' ================')
        # 1. Get all fitnesses 
        pop_fitness_results, fitnessDict = population_fitness(gen=gen, population=population, fitness_method=GAset['fitness_method'],
                                                     fitnessDict=fitnessDict, gmtLimit=GAset['gmtLimit'], threadPool_size=GAset['threadPool_size'])
        #all_GA_results.update(pop_fitness_results)
        all_GA_results = {**all_GA_results, **pop_fitness_results}
        # 2. Select fittest
        fitnessDF = selectFittest(topNum=GAset['select_fittest'], pop_fitness_results=pop_fitness_results, selection_method=GAset['selection_method'])
        # 3. Create/mutate children
        newPopulation = createChildren(GAset['children_per_generation'], fitnessDF, GAset['mutation_rate'],
                                       GAset['crossover_points'], GAset['crossover_locations'])
        if GAset['include_fittest_parents'] > 0:
            # When this is mixedTournament, selects from the parents that bred (regardless of whether they were the fittest in the whole population)
            #newPopulation.extend( fitnessDF.sort_values(by='fitness')[:GAset['include_fittest_parents']]['newBinary'].toList()) 
            newPopulation = newPopulation + fitnessDF.sort_values(by='fitness')[:GAset['include_fittest_parents']]['newBinary'].tolist()
        del population
        population = newPopulation
        del newPopulation 
    # Save
    save_results = GAset['save_results']
    if save_results!='No':
        import pickle 
        pickle.dump( all_GA_results, open( "Genetic_Algorithm/GA_Results/"+save_results+".pkl", "wb" ) )    
    return all_GA_results

def X2K_Web_GA_test(GA_train, GAset):
    all_GA_results={}
    fitnessDict={}
    for gen in range(GAset['generations']):
        population = GA_train.loc[GA_train['generation']==gen]['newBinary']
        print('================ GENERATION '+str(gen)+' ================')
        # 1. Get all fitnesses 
        pop_fitness_results, fitnessDict = population_fitness(gen=gen, population=population, fitness_method=GAset['fitness_method'],
                                                     fitnessDict=fitnessDict, gmtLimit=GAset['gmtLimit'], threadPool_size=GAset['threadPool_size'])
        #all_GA_results.update(pop_fitness_results)
        all_GA_results = {**all_GA_results, **pop_fitness_results}
    return all_GA_results

# GA_train = all_GA_results.copy()

run_name = 'GA_results_20gen_100inds'
GAset = {'GMT_train':"../X2K_Genetic_Algorithm/Validation/Perturbation_Data/GEO/Kinase_Perturbations_from_GEO_SUBSET1.80per.txt",
         'GMT_test':"../X2K_Genetic_Algorithm/Validation/Perturbation_Data/GEO/Kinase_Perturbations_from_GEO_SUBSET2.20per.txt",
                    'gmtLimit':False, 'initial_pop_size':100, 'generations':20, 'select_fittest':10, 'selection_method':'Fitness-proportional',
                    'fitness_method':'target_shuffled_difference', 'children_per_generation':95, 'mutation_rate':.01, 'breeding_variation':0,
                    'crossover_points':4, 'crossover_locations':'random','include_fittest_parents':5, 'threadPool_size':20,
         'save_results':run_name}
GAset_small = {'GMT_train':"../X2K_Genetic_Algorithm/Validation/Perturbation_Data/GEO/Kinase_Perturbations_from_GEO_SUBSET1.80per.txt",
         'GMT_test':"../X2K_Genetic_Algorithm/Validation/Perturbation_Data/GEO/Kinase_Perturbations_from_GEO_SUBSET2.20per.txt",
                    'gmtLimit':15, 'initial_pop_size':10, 'generations':10, 'select_fittest':4, 'selection_method':'Fitness-proportional',
                    'fitness_method':'target_shuffled_difference', 'children_per_generation':8, 'mutation_rate':.01, 'breeding_variation':0,
                    'crossover_points':4, 'crossover_locations':'random','include_fittest_parents':2, 'threadPool_size':20,
         'save_results':run_name}

def time_estimator(GAset):
    seconds_per_experiment = .5
    if GAset['gmtLimit']!=False:
        experiments = GAset['gmtLimit']
    else:
        experiments = 570
    minutes  = seconds_per_experiment * experiments * GAset['initial_pop_size'] * GAset['generations'] / GAset['threadPool_size'] /60
    hours = minutes/60
    print('GA will take: '+str(round(hours, 2))+' hours / '+str(round(minutes,2))+' minutes.')
time_estimator(GAset)
    




def GA_Train_Test(GAset):
    import time
   
    # Train GA
    start = time.time()
    GA_train = X2K_Web_GA_train(GAset) # GAset
    end = time.time()
    print("GA training took: "+str(round(end - start, 2))+"s")
    

    # Test GA  **** NEED TO HAVE TEST FUNCTION THAT USES POP FROM TRAINING
    start = time.time()
    GA_test = X2K_Web_GA_test(GA_train, GAset)
    end = time.time()
    print("GA testing took: "+str(round(end - start, 2))+"s")
    
    # Save
    import pickle
    GA_resultsDict = {'GA_train':GA_train, 'GA_test':GA_test, 'GA_settings':GAset}
    pickle.dump( GA_resultsDict, open( "Genetic_Algorithm/GA_Results/"+GAset['save_results']+".pkl", "wb" ) )
    
    return GA_resultsDict

GA_resultsDict = GA_Train_Test(GAset)

## GA Results Plots

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

import pickle
GA_resultsDict = pickle.load( open( "Genetic_Algorithm/GA_Results/GA_results_20gen_100inds.pkl", "rb" ) )
GA_train = GA_resultsDict['GA_train']

def plot_fitness(GA_train):
    
    dat = pd.DataFrame(GA_train).T
    dat[['generation','fitness']] = dat[['generation','fitness']].apply(pd.to_numeric)
    dat['unique_id'] = dat.index
    dat.groupby('generation').count()['fitness'] # Check that all gens have same # of individuals
    # Get peak fitness and add back to parent df
    dat_plot = dat.join(dat.groupby('generation')['fitness'].max(), on='generation', rsuffix='_peak').sort_values(by='unique_id')
    #peak_dat = dat.groupby('generation')['unique_id', 'fitness'].max().reset_index()
    #dat_plot['real_gen'] = dat_plot.index.str.split("_").str[1].str.strip('gen').astype(int)
    
    # # Plot
    # f, ax = plt.subplots(1, 1)
    # sns.pointplot(data=dat_plot, x='generation', y='fitness', label='Mean Fitness', color='limegreen', ax=ax)
    # sns.pointplot(data=dat_plot, x='generation', y='fitness_peak', label='Peak Fitness', color='forestgreen', markers='^', ax=ax)
    # # Add legend
    # ax.legend(handles=ax.lines[::len(dat_plot)+1], labels=["Fitness","Fitness"])
    # leg_handles = ax.get_legend_handles_labels()[0]
    # ax.legend(leg_handles, ['Blue', 'Orange'], title='New legend')
    
    
    
    grp = dat_plot.groupby('generation')
    fit_mean = grp['fitness'].mean().reset_index()
    fit_error = grp['fitness'].std().reset_index()
    fit_max = grp['fitness'].max().reset_index()
    
    f, ax = plt.subplots(1, 1)
    ax.errorbar(data=fit_mean, x='generation', y='fitness', yerr=fit_error['fitness'], label='Mean Fitness',
                 marker='o', markersize=4, color='limegreen', capsize=2)
    ax.errorbar(data=fit_max, x='generation', y='fitness', label='Peak Fitness', color='forestgreen', marker='^',  markersize=4)
    ax.legend(loc='lower right')
    plt.xticks(np.arange(min(fit_mean['generation']), max(fit_mean['generation'])+1, 2))
    plt.xlabel('Generation')
    plt.ylabel('Fitness')


    #ax.legend(handles=ax.lines[2:] , labels=["Mean Fitness","Peak Fitness" ])
    

## Plot top individual's performance

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def get_top_individual(GA_train):
    dat = pd.DataFrame(GA_train).T
    dat[['generation','fitness']] = dat[['generation','fitness']].apply(pd.to_numeric)
    top_individual = dat.sort_values(by='fitness', ascending=False).iloc[0]
    top_individual['unique_ID'] = top_individual.name
    return top_individual 

def get_average_individual(GA_train):
    dat = pd.DataFrame(GA_train).T
    dat[['generation','fitness']] = dat[['generation','fitness']].apply(pd.to_numeric)
    average_start_fitness = dat[dat['generation']==0]['fitness'].mean() 
    average_individual = dat.iloc[(dat['fitness']-average_start_fitness).abs().argsort()].iloc[0,:]
    return average_individual

individual=top_individual
individual=average_individual

 
def KDE_multiplot(individual, scaledRanks=True, saveFig=False, supTitle=''):
    # Get ranks
    shuffled_ranks = individual['shuffled_mean_ranks']
    target_ranks =   individual['target_ranks']
    zScores = values_to_zscores( pd.DataFrame(individual['KEA_results']) )
    ranks_matrix = values_to_ranks(zScores)
    
    # Plot
    #ax = plt.subplots(111)
    plt.figure()
    DFstack = ranks_matrix.stack().reset_index()
    DFstack.columns = ['Kinase','Experiment','Rank']
    if scaledRanks==True:
        DFstack = scaled_ranks(DFstack) 
    ## Null distribution (all kinase ranks)
    g0 = sns.distplot( DFstack['Rank'], label='All Kinases',rug=False, hist=False, norm_hist=True ).set_xlim(0,1) 
    
    ## Target Kinases Only
    g1 = sns.distplot( target_ranks, label='Target Kinases',rug=False, hist=True, norm_hist=True ).set_xlim(0,1) 
    
    ## Shuffled targets
    g2 = sns.distplot( shuffled_ranks, label='Shuffled Kinases',rug=False, hist=True, norm_hist=True,
                  kde_kws={"linestyle":"--"}).set_xlim(0,1) 
    plt.legend(loc='upper right')
    plt.xlim(0,1)
    plt.title('X2K GA Optimized Parameters')
    plt.xlabel('Scaled Ranks')
     
    plt.suptitle(supTitle)
    if saveFig==True:
        plt.draw()
        f.savefig('Figures/Rank_Distribution_Plots/'+ supTitle+'.png')
    return f
 
 
KDE_multiplot(individual=get_top_individual(GA_train), scaledRanks=True, saveFig=False, supTitle='Optimized X2K Results')
KDE_multiplot(individual=get_average_individual(GA_train), scaledRanks=True, saveFig=False, supTitle='Optimized X2K Results')



## Assess Optimal Parameters

In [ ]:

optimal_params = binary_to_parameters(top_individual['newBinary'])
for key in optimal_params:
    print(key+' : '+str(optimal_params[key]['selection']))